In [ ]:
from mlflow import MlflowClient
import mlflow
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from joblib import Parallel, delayed
from itertools import product
from Source.preprocess_data import *  ## import all functions from preprocess_data.py
from Source.postprocess_data import * ## import all functions from postprocess_data.py
from Source.utils import *  ## import all functions from utils.py
import nltk


client = MlflowClient(tracking_uri="http://localhost:8080")

# Prétraitement du dataset


## Extraction d'un faible pourcentage de tweets

In [ ]:
df = pd.read_csv('https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip',
                header=None,
                compression='zip',
                encoding='cp1252')

df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

sample_df, _ = train_test_split(df, test_size=0.9, random_state=42, stratify=df['target'])
sample_df = sample_df.reset_index(drop=True)
print(f"Sample size: {sample_df.shape[0]} rows")
# On ne garde que les colonnes 'target' et 'text'
sample_df = sample_df[['target', 'text']]
sample_df.to_csv('Data/raw_data.csv', index=False)


## Normalisation du texte

In [ ]:
# Normalisation du texte
sample_df['text'] = sample_df['text'].apply(lambda x: normalize_text(x))
# Enregistrement du jeu de données pré-traité
sample_df.to_csv('Data/normalized_data.csv', index=False)
print(f"Normalized {sample_df.shape[0]} rows")


## Tokenisation

In [ ]:
#Stopwords
nltk.download('stopwords')
#Vocabulaire
nltk.download('words')
#Punctuation
nltk.download('punkt_tab')
#Wordnet
nltk.download('wordnet')
#POS-tagging
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
sample_df.sample(10, random_state=42)

In [ ]:
sample_df["target"] = sample_df["target"].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
sample_df.info()

In [ ]:
sns.countplot(x='target',hue='target', data=sample_df)

Dataset équilibré, pas besoin de rééquilibrer la cible.  

## Experimentation sur l'impact du feature engineering (tokenisation, stemming/lemmatisation, vectorisation) 

### Mise en place de l'environnement MLFlow

In [ ]:
# Provide an Experiment description that will appear in the UI

mlflow.set_experiment("Tokenization experiment")
exp_id = mlflow.get_experiment_by_name("Tokenization experiment").experiment_id

experiment_description = (
    "Cette experience contient les différents tests pour le <<modèle sur mesure simple>>. "
    "Le but est d'évaluer un modèle simple permettant d'évaluer les sentiments dans les tweets à partir d'un modèle de régression simple."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "simple-regression",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)


### Fonction de run

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix 
from nltk.corpus import stopwords  
from nltk.tokenize import TweetTokenizer, WordPunctTokenizer, RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer, LancasterStemmer

from tqdm import tqdm
tqdm.pandas()



In [ ]:

def model1_experiment(
    df: pd.DataFrame,
    params: dict
):
    """
    Lance une expérience MLflow avec les paramètres fournis.

    Paramètres attendus dans params :
        - stop_words : liste de stopwords ou None (default=None)
        - tokenizer : fonction de tokenisation (default=None)
        - stem_lem_func : fonction de stemming/lemmatisation (default=None)
        - vectorizertype : type de vectoriseur, CountVectorizer ou TfidfVectorizer (default=CountVectorizer)
        - min_df : int, fréquence minimale des mots (default=1)
        - ngram_range : tuple, plage des n-grammes (default=(1,1)), 
        - model_name : nom du modèle pour enregistrement dans MLflow (default="log_regression_v1")
    """
    # Paramètres par défaut
    default_params = {

        "stop_words": None,
        "tokenizer": RegexpTokenizer(r'\w+').tokenize,
        "stem_lem_func": WordNetLemmatizer().lemmatize,
        "vectorizertype": "CountVectorizer",
        "min_df": 1,
        "ngram_range": (1, 1), 
        }
    # Met à jour les paramètres manquants
    for k, v in default_params.items():
        if k not in params:
            params[k] = v


    
    with mlflow.start_run():
        mlflow.log_params(params)

        X_raw = df['text']
        y = df['target']
        X_train, X_val, y_train, y_val = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)

        # Prétraitement des textes
        ## Jeu de données d'entraînement
        X_train_stem = X_train.apply(lambda x:preprocess_text_simple(
                                                                    x,
                                                                    tokenizer=params["tokenizer"],
                                                                    stem_lem_func=params["stem_lem_func"]
                                                                    )
                                    )
        ## Jeu de données de validation
        X_val_stem = X_val.apply(lambda x:preprocess_text_simple(
                                                                x,
                                                                tokenizer=params["tokenizer"],
                                                                stem_lem_func=params["stem_lem_func"]
                                                                )
                                )

        # Selection du vectorizer
        match params["vectorizertype"]:
            case "CountVectorizer":
                vectorizer = CountVectorizer(min_df=params["min_df"], ngram_range=params["ngram_range"], stop_words=params["stop_words"])
            case "TfidfVectorizer":
                vectorizer = TfidfVectorizer(min_df=params["min_df"], ngram_range=params["ngram_range"], stop_words=params["stop_words"])
            case _:
                raise ValueError("Invalid vectorizer type")
        # Création du pipeline
        pipe = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', LogisticRegression(C=1.0, max_iter=1000))
        ])
        # Entraînement du modèle
        pipe.fit(X_train_stem, y_train)
        # Prédictions sur le jeu de validation
        y_val_pred = pipe.predict(X_val_stem)
        y_val_proba = pipe.predict_proba(X_val_stem)[:, 1]
        # Évaluation du modèle
        output_dict = postprocess_model_output(y_val, y_val_pred, y_val_proba) # voir postprocess_data.py
        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_val_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        # Enregistrement du modèle dans MLflow
        mlflow.sklearn.log_model(pipe, "model")
        


### Test baseline 

In [ ]:
# Baseline avec des paramètres par défaut

params = {}
model1_experiment(df=sample_df, params=params)

### Grille de paramètres 

In [ ]:

param_grid = {
        "stop_words": [None, stopwords.words('english')],
        "tokenizer": [RegexpTokenizer(r'\w+').tokenize, TweetTokenizer().tokenize, WordPunctTokenizer().tokenize],
        "stem_lem_func": [WordNetLemmatizer().lemmatize, LancasterStemmer().stem, PorterStemmer().stem],
        "vectorizertype": ["CountVectorizer", "TfidfVectorizer"],
        "min_df": [1, 5, 10],
        "ngram_range": [(1, 1), (1, 2), (1, 3)]
    }

In [ ]:
from itertools import product

param_combinations = list(product(*param_grid.values()))
param_names = list(param_grid.keys())

print(f"Total combinations to try: {len(param_combinations)}")

### Lancement de la grille de paramètres 

In [ ]:
# for i, comb in enumerate(param_combinations):
#     params = dict(zip(param_names, comb))
#     print(f"Running combination {i+1}/{len(param_combinations)}: {params}")
#     model1_experiment(df=sample_df, params=params)

### Sélection du meilleur modèle

In [ ]:
# Sélection du meilleur modèle
# Reset client ? 
client = MlflowClient(tracking_uri="http://localhost:8080")
experiment_id = mlflow.get_experiment_by_name("Tokenization experiment").experiment_id
runs = client.search_runs(experiment_id)

# Métrique pour sélectionner le meilleur modèle
metric_to_optimize = "Accuracy" # liste des métriques enregistrées dans postprocess_data.py ou sur l'UI MLflow
best_run = max(runs, key=lambda run: run.data.metrics.get(metric_to_optimize, float('-inf')))
print(f"Best run ID: {best_run.info.run_id} with metrics:")
for key, value in best_run.data.metrics.items():
    print(f"{key}: {value}")
print(f"Best run parameters:")
for key, value in best_run.data.params.items():
    print(f"{key}: {value}")

# Enregistrement du meilleur modèle
best_model_uri = f"runs:/{best_run.info.run_id}/model"
registered_model_name = "log_regression_model"
registered_model = mlflow.register_model(best_model_uri, registered_model_name)
# Enregistrement des paramètres sous forme de tags dans le modèle enregistré
for key, value in best_run.data.params.items():
    print(f"Setting tag {key} = {value} in registered model")
    client.set_model_version_tag(
        name=registered_model_name,
        version=str(registered_model.version),
        key=str(key),
        value=str(value))
    


## Les paramètres de tokenization optimaux ont été établis

In [ ]:
mlflow.set_experiment("Logreg hyperparameter exepriment")
exp_id = mlflow.get_experiment_by_name("Logreg hyperparameter exepriment").experiment_id

experiment_description = (
    "Cette experience contient les différents tests pour le <<modèle sur mesure simple>>. "
    "Le but est d'évaluer un modèle simple permettant d'évaluer les sentiments dans les tweets à partir d'un modèle de régression simple."
    "Ici on va chercher à optimiser les hyperparamètres de la régression logistique."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "simple-regression",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

In [ ]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tqdm

# Data
X_raw = sample_df['text']
y = sample_df['target']
X_train, X_val, y_train, y_val = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)

# Prétraitement des textes
## Jeu de données d'entraînement
print("Preprocessing training data...")
X_train_stem = X_train.apply(lambda x:preprocess_text_simple(
                                                                    x,
                                                                    tokenizer=TweetTokenizer().tokenize,
                                                                    stem_lem_func=WordNetLemmatizer().lemmatize
                                                                    )
                                    )
## Jeu de données de validation
print("Preprocessing validation data...")
X_val_stem = X_val.apply(lambda x:preprocess_text_simple(
                                                                x,
                                                                tokenizer=TweetTokenizer().tokenize,
                                                                stem_lem_func=WordNetLemmatizer().lemmatize
                                                                )
                                )
print("Preprocessing done.")
# Définition de la fonction objective pour Optuna
def logreg_eval(trial):
    # Hyperparamètres
    C = trial.suggest_float("C", 1e-3, 1e2, log=True)   # numérique (régularisation)
    max_iter = trial.suggest_int("max_iter", 10, 1000, log=True)  # numérique (itérations)
    tol = trial.suggest_float("tol", 1e-5, 1e-1, log=True)  # numérique (tolérance)
    penalty = trial.suggest_categorical("penalty", ["l1", "l2", "elasticnet"])  # catégoriel
    l1_ratio = trial.suggest_float("l1_ratio", 0, 1) if penalty == "elasticnet" else None
    solver = trial.suggest_categorical("solver", ["liblinear", "saga"])  # catégoriel
    

    # Attention : tous les solveurs ne supportent pas toutes les pénalités
    if penalty == "elasticnet" and solver != "saga":
        raise optuna.exceptions.TrialPruned()
    if penalty == "l1" and solver not in ["liblinear", "saga"]:
        raise optuna.exceptions.TrialPruned()

    with mlflow.start_run(nested=True):
        mlflow.log_params(params={
            'C':C,               
            'Max iterations': max_iter,
            'tolerance': tol, 
            'penalty': penalty, 
            'l1_ratio': l1_ratio, 
            'Solver': solver 
        })
        # Modèle
        pipe = Pipeline([
            ('vectorizer', TfidfVectorizer(min_df=5, ngram_range=(1,3),stop_words=None)),
            ('classifier', LogisticRegression(C=C, 
                                              max_iter=max_iter, 
                                              tol=tol, 
                                              penalty=penalty, 
                                              solver=solver, 
                                              l1_ratio=l1_ratio))
        ])
        # Entraînement du modèle
        pipe.fit(X_train_stem, y_train)
        # Prédictions sur le jeu de validation
        y_val_pred = pipe.predict(X_val_stem)
        y_val_proba = pipe.predict_proba(X_val_stem)[:, 1]
        # Évaluation du modèle
        output_dict = postprocess_model_output(y_val, y_val_pred, y_val_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_val_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        # Enregistrement du modèle dans MLflow
        mlflow.sklearn.log_model(pipe, "model")
        acc = output_dict["Accuracy"]
    return acc


# Création de l'étude Optuna et optimisation
print("Starting hyperparameter optimization with Optuna...")
print("Setting up MLflow experiment...")
mlflow.set_experiment("optuna_logreg_experiment")
exp_id = mlflow.get_experiment_by_name("optuna_logreg_experiment").experiment_id

experiment_description = (
    "Cette experience contient les différents tests pour le <<modèle sur mesure simple>>. "
    "Le but est d'évaluer un modèle simple permettant d'évaluer les sentiments dans les tweets à partir d'un modèle de régression simple."
    "Ici on va chercher à optimiser les hyperparamètres de la régression logistique."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "simple-regression",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)



# Lancement de l'optimisation avec Optuna
print("Starting optimization trials...")
with mlflow.start_run(run_name="optuna_logreg_optimization"):
    study = optuna.create_study(direction="maximize")
    study.optimize(logreg_eval, n_trials=50)

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_accuracy", study.best_value)

print("Optimization completed.")


In [ ]:
client = MlflowClient(tracking_uri="http://localhost:8080")
experiment_id = mlflow.get_experiment_by_name("optuna_logreg_experiment").experiment_id
runs = client.search_runs(experiment_id)

# Métrique pour sélectionner le meilleur modèle
metric_to_optimize = "Accuracy" # liste des métriques enregistrées dans postprocess_data.py ou sur l'UI MLflow
best_run = max(runs, key=lambda run: run.data.metrics.get(metric_to_optimize, float('-inf')))
print(f"Best run ID: {best_run.info.run_id} with metrics:")
for key, value in best_run.data.metrics.items():
    print(f"{key}: {value}")
print(f"Best run parameters:")
for key, value in best_run.data.params.items():
    print(f"{key}: {value}")

# Enregistrement du meilleur modèle
best_model_uri = f"runs:/{best_run.info.run_id}/model"
registered_model_name = "log_regression_model_opt"
registered_model = mlflow.register_model(best_model_uri, registered_model_name)
# Enregistrement des paramètres sous forme de tags dans le modèle enregistré
for key, value in best_run.data.params.items():
    print(f"Setting tag {key} = {value} in registered model")
    client.set_model_version_tag(
        name=registered_model_name,
        version=str(registered_model.version),
        key=str(key),
        value=str(value))